In [1]:
import keras

(X_train, _), (X_test, _) = keras.datasets.mnist.load_data()

ModuleNotFoundError: No module named 'keras'

# Simple Autoencoder

We start with a simple autoencoder based on a fully connected layers. One hidden layer handles the encoding, and the output layer handles the decoding.
Each of the input images is flatten to an array of 784 (=28×28) data points. This is then compressed into 32 data points by the fully connected layer.

In [ ]:
inputs  = Input(shape=(784,))           # 28*28 flatten
enc_fc  = Dense( 32, activation='relu') # to 32 data points
encoded = enc_fc(inputs)

Then, we decode the encoded data to the original 784 data points. The sigmoid will return values between 0 and 1 for each pixel (intensity).

In [ ]:
dec_fc  = Dense(784, activation='sigmoid') # to 784 data points
decoded = dec_fc(encoded)

This whole processing becomes the trainable autoencoder model.

In [ ]:
autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

We preprocess the MNIST image data so that image data are normalized between 0 and 1.

In [ ]:
def preprocess(x):
    x = x.astype('float32') / 255.
    return x.reshape(-1, np.prod(x.shape[1:])) # flatten
X_train = preprocess(X_train)
X_test  = preprocess(X_test)

We also split the train data into a train set and a validation set.

In [ ]:
X_train, X_valid = train_test_split(X_train, test_size=500)

We train the autoencoder which compress the input image and then restore to the original size. As such, our training data and label data are both the same image data.

In [ ]:
autoencoder.fit(X_train, X_train, # data and label are the same
                epochs=50, 
                batch_size=128, 
                validation_data=(X_valid, X_valid))

By training an autoencoder, we are really training both the encoder and the decoder at the same time.
We can build an encoder and use it to compress MNIST digit images.

In [ ]:
encoder = Model(inputs, encoded)
X_test_encoded = encoder.predict(X_test)

Let’s also build a decoder so that we can decompress the compressed image to the original image size. The decoder takes 32 data points as its input (the size of encoded data).

In [ ]:
decoder_inputs = Input(shape=(32,))
decoder = Model(decoder_inputs, dec_fc(decoder_inputs))
# decode the encoded test data
X_test_decoded = decoder.predict(X_test_encoded)

# Convolutional Autoencoder

In [ ]:
def make_convolutional_autoencoder():
    # encoding
    inputs = Input(shape=(28, 28, 1))
    x = Conv2D(16, 3, activation='relu', padding='same')(inputs)
    x = MaxPooling2D(padding='same')(x)
    x = Conv2D( 8, 3, activation='relu', padding='same')(x)
    x = MaxPooling2D(padding='same')(x)
    x = Conv2D( 8, 3, activation='relu', padding='same')(x)
    encoded = MaxPooling2D(padding='same')(x)    
    
    # decoding
    x = Conv2D( 8, 3, activation='relu', padding='same')(encoded)
    x = UpSampling2D()(x)
    x = Conv2D( 8, 3, activation='relu', padding='same')(x)
    x = UpSampling2D()(x)
    x = Conv2D(16, 3, activation='relu')(x) # <= padding='valid'!
    x = UpSampling2D()(x)
    decoded = Conv2D(1, 3, activation='sigmoid', padding='same')(x)
    
    # autoencoder
    autoencoder = Model(inputs, decoded)
    autoencoder.compile(optimizer='adam', 
                        loss='binary_crossentropy')
    return autoencoder
# create a convolutional autoencoder
autoencoder = make_convolutional_autoencoder()

Now, we reshape the image data to the format the convolutional autoencoder expects for training.

In [ ]:
# reshape the flattened images to 28x28 with 1 channel
X_train = X_train.reshape(-1, 28, 28, 1)
X_valid = X_valid.reshape(-1, 28, 28, 1)
X_test  = X_test.reshape(-1, 28, 28, 1)
autoencoder.fit(X_train, X_train, 
                epochs=50, 
                batch_size=128, 
                validation_data=(X_valid, X_valid))

We just want to see the quality of compression/decompression, for which we do not need to build separate encoder and decoder models. So, we simply feed forward test images to see how the restored digits look like.

In [ ]:
X_test_decoded = autoencoder.predict(X_test)

# Noise reduction

In [ ]:
def add_noise(x, noise_factor=0.2):
    x = x + np.random.randn(*x.shape) * noise_factor
    x = x.clip(0., 1.)
    return x
    
X_train_noisy = add_noise(X_train)
X_valid_noisy = add_noise(X_valid)
X_test_noisy  = add_noise(X_test)

We train a new autoencoder with the noisy data as input and the original data as expected output.

In [ ]:
autoencoder = make_convolutional_autoencoder()
autoencoder.fit(X_train_noisy, X_train, 
                epochs=50, 
                batch_size=128, 
                validation_data=(X_valid_noisy, X_valid))

During the training, the autoencoder learns to extract important features from input images and ignores the image noises because the labels have no noises.
Let’s pass the noisy test images to the autoencoder to see the restored images.

In [ ]:
X_test_decoded = autoencoder.predict(X_test_noisy)